Imports

In [ ]:
import os
import numpy as np
import cv2
from time import time
import logging
import matplotlib.pyplot as plt
import random
import pandas as pd
import math

from random import seed
from random import randrange

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import confusion_matrix, classification_report,precision_score,f1_score,recall_score,accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

In [ ]:
#dictionary to save the results
LBPH_results = dict({'bioid':None,'lfw':None})

In [ ]:
train_sizes=np.empty(5)
train_sizes[:]=np.NaN
train_scores=np.empty([5, 10])
train_scores[:]=np.NaN
test_scores=np.empty([5, 10])
test_scores[:]=np.NaN
fit_times=np.empty([5, 10])
fit_times[:]=np.NaN
score_times=np.empty([5, 10])
score_times[:]=np.NaN
lbph_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image,minNeighbors=6,minSize=(75, 75)) #parameters change depending the dataset
    if (len(face)==1): #if 1 face is detected
        (x,y,w,h) = face[0]
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        
        return False, "zero"
    else: #if more than 1 faces are detected, the biggest is kept (the smaller ones usually belong in the background)
        biggest=0
        pos=0
        for i,(x,y,w,h) in enumerate(face):
            if biggest<(h*w):
                biggest=h*w
                pos=i
        return True, image[y:y+w, x:x+h], face[pos]

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image)#,scaleFactor=1.2, minNeighbors=5)
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        '''
        for (x,y,w,h) in face: 
            cv2.rectangle(image, (x,y), (x+w,y+h), (255,255,255), 3)
        plt.imshow(image)
        plt.show()
        '''
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        return False, "zero"
    else: #if more than 1 faces are detected
        return False, "more"

In [ ]:
def eye_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image, (200, 200), interpolation=cv2.INTER_LINEAR)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye.xml')
    eyes = haar_classifier.detectMultiScale(image, minNeighbors=4)#, minSize=(30, 30))
    if (len(eyes)==2): #if 2 eyes are detected
        #(x,y,w,h) = face[0]
        return True, eyes
    else:      #if the number of eyes found is different from 2
        return False, "more or less"

In [ ]:
datapath=('C:/Users/Thomas/Documents/Datasets/ExtYale(640x480)/')
min_faces=30
scores=[]

names=["accuracy","macro_avg_precision","macro_avg_recall","macro_avg_f1_score",
       "micro_avg_precision","micro_avg_recall","micro_avg_f1_score",
       "weighted_avg_precision","weighted_avg_recall","weighted_avg_f1_score",
       "fit_time"]

df = pd.DataFrame(index=names)

target_names=[] #all the names are saved
total_photos_seen=0 
n_classes=0
folders = os.listdir(datapath)
for folder in folders:
    label = os.path.basename(folder)
    training_images_path = datapath + '/' + folder
    num_of_faces = len(os.listdir(training_images_path))
    #if num_of_faces>=min_faces:   #people with low number of faces are skipped
    target_names.append(label)
    n_classes=n_classes+1
    faces_per_person=0
    for image in os.listdir(training_images_path):
        total_photos_seen=total_photos_seen+1
        image_path = training_images_path + '/' + image
        training_image = cv2.imread(image_path)
        #face=cv2.cvtColor(training_image,cv2.COLOR_BGR2GRAY)
        #resized=cv2.resize(face,(224,224),interpolation=cv2.INTER_AREA)
        result_im=face_detection(training_image) #this function returns TRUE/FALSE, the cut image and the coordinates of where the cut was made
        if result_im[0]: #if TRUE (face detected) then the cut face goes into the array
            cut_face=result_im[1]
            faces.append(cut_face)
            labels.append(n_classes)

        '''
        if faces_per_person==45:
                break
        '''       

In [ ]:
new_faces=[]
new_labels=[]

In [ ]:
rotated_faces=0
for i in range(len(faces)):
    training_image = cv2.imread(faces[i])
    result_im=eye_detection(training_image) #finding the place of the eyes
    
    if result_im[0]:  #process of finding the angle of the two eyes, and alighnment
        rotated_faces=rotated_faces+1
        (x1,y1,w1,h1)=result_im[1][0]
        (x2,y2,w2,h2)=result_im[1][1]
        eye1_center_x=x1+(w1/2)
        eye1_center_y=y1+(h1/2)
        eye2_center_x=x2+(w2/2)
        eye2_center_y=y2+(h2/2)
        if (eye1_center_x>eye2_center_y):
            deltaY = eye1_center_y - eye2_center_y 
            deltaX = eye1_center_x - eye2_center_x
        else:
            deltaY = eye2_center_y - eye1_center_y 
            deltaX = eye2_center_x - eye1_center_x
        my_radians = math.atan2(deltaY , deltaX)
        my_degrees = math.degrees(my_radians)

        #rotation
        rows, cols = training_image.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), my_degrees, 1)
        img_rotated = cv2.warpAffine(training_image, M, (cols,rows))
        '''
        font = cv2.FONT_HERSHEY_SIMPLEX
        plt.text(0,0,i)
        plt.imshow(img_rotated)
        plt.show()
        '''

        #scaling (all images will become 200x200)
        #if before the conversion the image is smaller than 200x200 INTER_LINEAR will be used, else INTER_AREA will be used
        image_size=rows*cols        
        if (image_size<=40000):
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_LINEAR)
        else:
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_AREA)

        #normalization
        norm_img = np.zeros((200, 200))
        norm_img = cv2.normalize(scaled_image, norm_img, 0, 255, cv2.NORM_MINMAX)

        final_image = cv2.cvtColor(norm_img, cv2.COLOR_BGR2GRAY)

        new_faces.append(final_image)
        new_labels.append(labels[i])
        
faces=new_faces
labels=new_labels

In [ ]:
kf=StratifiedKFold(n_splits=10)

for k_f,idx in enumerate(kf.split(X=faces, y=labels)):
    train_idx,test_idx=idx[0],idx[1]
    X_train=faces[train_idx]
    y_train=labels[train_idx]

    X_test=faces[test_idx]
    y_test=labels[test_idx]

    print("Fold Number : ",k_f+1)  
    #in every loop, different folds are used
    #in every loop different models are trained with different training set sizes (in the end with all the data), in order to find the data for the learning curve
    #for this to happen, a random part of each fold is selected in every loop
    
    #14% size training===========================================================================================================
    X_train_10, X, y_train_10, y_ = train_test_split(X_train, y_train, train_size=0.22,shuffle=True,stratify=y_train)
    X_, X_test_10, y_, y_test_10 = train_test_split(X_test, y_test, test_size=0.22,shuffle=True,stratify=y_test)

    lbph_learning_curve[1][0]=len(X_train_10)#setting train_size
    
    print("Fitting the recognizer to the training set=====14% size")
    t0 = time()
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train_10, y_train_10)
    fit_time=time() - t0
    print("done in %0.3fs" %fit_time )
    lbph_learning_curve[4][0][k_f]=fit_time#setting fit_time
    
    print("Finding train scores=====14% size")
    train_preds=[]
    for image in X_train_10:
        prediction = model.predict(image)
        train_preds.append(prediction[0])
    lbph_learning_curve[2][0][k_f]=accuracy_score(y_train_10, train_preds)#setting train_score
    
    print("Predicting people's names on the test set=====14% size")
    t0 = time()
    y_pred=[]
    for image in X_test_10:
        prediction = model.predict(image)
        y_pred.append(prediction[0])
    score_time=time() - t0
    print("done in %0.3fs" %score_time)
    lbph_learning_curve[3][0][k_f]=accuracy_score(y_test_10, y_pred)#setting test_score
    lbph_learning_curve[5][0][k_f]=score_time#setting score_times
    
    
    
    #32.5% size training===========================================================================================================
    X_train_325, X, y_train_325, y_ = train_test_split(X_train, y_train, train_size=0.325,shuffle=True,stratify=y_train)
    X_, X_test_325, y_, y_test_325 = train_test_split(X_test, y_test, test_size=0.325,shuffle=True,stratify=y_test)
    
    lbph_learning_curve[1][1]=len(X_train_325)#setting train_size
    
    print("Fitting the recognizer to the training set=====32.5% size")
    t0 = time()
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train_325, y_train_325)
    fit_time=time() - t0
    print("done in %0.3fs" %fit_time )
    lbph_learning_curve[4][1][k_f]=fit_time#setting fit_time
    
    print("Finding train scores=====32.5% size")
    train_preds=[]
    for image in X_train_325:
        prediction = model.predict(image)
        train_preds.append(prediction[0])
    lbph_learning_curve[2][1][k_f]=accuracy_score(y_train_325, train_preds)#setting train_score
    
    print("Predicting people's names on the test set=====32.5% size")
    t0 = time()
    y_pred=[]
    for image in X_test_325:
        prediction = model.predict(image)
        y_pred.append(prediction[0])
    score_time=time() - t0
    print("done in %0.3fs" %score_time)
    lbph_learning_curve[3][1][k_f]=accuracy_score(y_test_325, y_pred)#setting test_score
    lbph_learning_curve[5][1][k_f]=score_time#setting score_times
    
    #55% size training===========================================================================================================
    X_train_55, X, y_train_55, y = train_test_split(X_train, y_train, train_size=0.55,shuffle=True,stratify=y_train)
    X_, X_test_55, y_, y_test_55 = train_test_split(X_test, y_test, test_size=0.55,shuffle=True,stratify=y_test)

    lbph_learning_curve[1][2]=len(X_train_55)#setting train_size
    
    print("Fitting the recognizer to the training set=====55% size")
    t0 = time()
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train_55, y_train_55)
    fit_time=time() - t0
    print("done in %0.3fs" %fit_time )
    lbph_learning_curve[4][2][k_f]=fit_time#setting fit_time
    
    print("Finding train scores=====55% size")
    train_preds=[]
    for image in X_train_55:
        prediction = model.predict(image)
        train_preds.append(prediction[0])
    lbph_learning_curve[2][2][k_f]=accuracy_score(y_train_55, train_preds)#setting train_score
    
    print("Predicting people's names on the test set=====55% size")
    t0 = time()
    y_pred=[]
    for image in X_test_55:
        prediction = model.predict(image)
        y_pred.append(prediction[0])
    score_time=time() - t0
    print("done in %0.3fs" %score_time)
    lbph_learning_curve[3][2][k_f]=accuracy_score(y_test_55, y_pred)#setting test_score
    lbph_learning_curve[5][2][k_f]=score_time#setting score_times
    
    #77.5% size training===========================================================================================================
    X_train_775, X, y_train_775, y_ = train_test_split(X_train, y_train, train_size=0.77,shuffle=True,stratify=y_train)
    X_, X_test_775, y_, y_test_775 = train_test_split(X_test, y_test, test_size=0.77,shuffle=True,stratify=y_test)
    
    lbph_learning_curve[1][3]=len(X_train_775)#setting train_size
    
    print("Fitting the recognizer to the training set=====77.5% size")
    t0 = time()
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train_775, y_train_775)
    fit_time=time() - t0
    print("done in %0.3fs" %fit_time )
    lbph_learning_curve[4][3][k_f]=fit_time#setting fit_time
    
    print("Finding train scores=====77.5% size")
    train_preds=[]
    for image in X_train_775:
        prediction = model.predict(image)
        train_preds.append(prediction[0])
    lbph_learning_curve[2][3][k_f]=accuracy_score(y_train_775, train_preds)#setting train_score
    
    print("Predicting people's names on the test set=====77.5% size")
    t0 = time()
    y_pred=[]
    for image in X_test_775:
        prediction = model.predict(image)
        y_pred.append(prediction[0])
    score_time=time() - t0
    print("done in %0.3fs" %score_time)
    lbph_learning_curve[3][3][k_f]=accuracy_score(y_test_775, y_pred)#setting test_score
    lbph_learning_curve[5][3][k_f]=score_time#setting score_times
    
    #100% size=========================================================================================================
    
    lbph_learning_curve[1][4]=len(X_train)#setting train_size
    
    print("Fitting the recognizer to the training set========full size")
    t0 = time()
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train, y_train)
    fit_time=time() - t0
    print("done in %0.3fs" %fit_time )
    lbph_learning_curve[4][4][k_f]=fit_time#setting fit_time
    
    print("Finding train scores=====full size")
    train_preds=[]
    for image in X_train:
        prediction = model.predict(image)
        train_preds.append(prediction[0])
    lbph_learning_curve[2][4][k_f]=accuracy_score(y_train, train_preds)#setting train_score
    

    print("Predicting people's names on the test set=====full size")
    t0 = time()
    y_pred=[]
    for image in X_test:
        prediction = model.predict(image)
        y_pred.append(prediction[0])
    score_time=time() - t0
    print("done in %0.3fs" %score_time)
    lbph_learning_curve[3][4][k_f]=accuracy_score(y_test, y_pred)#setting test_score
    lbph_learning_curve[5][4][k_f]=score_time#setting score_times

    report=classification_report(y_test, y_pred,output_dict=True)
    accuracy=report['accuracy']
    macro_avg_precision=report['macro avg']['precision']
    macro_avg_recall=report['macro avg']['recall']
    macro_avg_f1_score=report['macro avg']['f1-score']
    micro_avg_precision=precision_score(y_test, y_pred, average='micro')
    micro_avg_recall=recall_score(y_test, y_pred, average='micro')
    micro_avg_f1_score=f1_score(y_test, y_pred, average='micro')
    weighted_avg_precision=report['weighted avg']['precision']
    weighted_avg_recall=report['weighted avg']['recall']
    weighted_avg_f1_score=report['weighted avg']['f1-score']
    
    #scores.append((accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score))
    df[k_f+1]=[accuracy,macro_avg_precision,macro_avg_recall,macro_avg_f1_score,
               micro_avg_precision,micro_avg_recall,micro_avg_f1_score,
               weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
               fit_time]
    
print("Loop Done!!!")

Saving in pickle file

In [ ]:
#with open('lbph_bioid.p', 'wb') as fp:
#        pickle.dump(df_bioid, fp, protocol=pickle.HIGHEST_PROTOCOL)
df.to_pickle('lbph')
#with open('lbph_learning_curve_bioid.p', 'wb') as fp:
#        pickle.dump(lbph_learning_curve_bioid, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('lbph_learning_curve.p', 'wb') as fp:
        pickle.dump(lbph_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)